In [20]:
import os

import numpy as np
import torch

from seq2seq_translation.inference import BeamSearchSequenceGenerator
from seq2seq_translation.tokenization.sentencepiece_tokenizer import SentencePieceTokenizer
from seq2seq_translation.run import _fix_model_state_dict
from seq2seq_translation.attention import AttentionType
from seq2seq_translation.rnn import EncoderRNN, AttnDecoderRNN, EncoderDecoder

In [10]:
torch.random.manual_seed(1234)
np.random.seed(1234)

In [11]:
source_tokenizer = SentencePieceTokenizer(model_prefix='/Users/adam.amster/2024-10-03-sequence_to_sequence_models_1/tokenizer/30000/en30000')
target_tokenizer = SentencePieceTokenizer(model_prefix='/Users/adam.amster/2024-10-03-sequence_to_sequence_models_1/tokenizer/30000/fr30000')

In [12]:
def construct_model_attention():
	encoder = EncoderRNN(
		input_size=source_tokenizer.processor.vocab_size(),
		hidden_size=1000,
		bidirectional=True,
		pad_idx=source_tokenizer.processor.pad_id(),
		embedding_dim=1000,
		num_layers=4,
	)

	decoder = AttnDecoderRNN(
		hidden_size=1000,
		attention_size=1000,
		output_size=target_tokenizer.processor.vocab_size(),
		encoder_bidirectional=True,
		max_len=200,
		attention_type=AttentionType.CosineSimilarityAttention,
		encoder_output_size=encoder.output_size,
		pad_idx=source_tokenizer.processor.pad_id(),
		num_embeddings=target_tokenizer.processor.vocab_size(),
		sos_token_id=source_tokenizer.processor.bos_id(),
		embedding_dim=1000,
		num_layers=4,
		eos_token_id=target_tokenizer.processor.eos_id()
	)
	
	encoder.load_state_dict(
		_fix_model_state_dict(torch.load('/Users/adam.amster/2024-10-03-sequence_to_sequence_models_1/weights/attention/encoder.pt', map_location='cpu'))
	)
	decoder.load_state_dict(
		_fix_model_state_dict(torch.load('/Users/adam.amster/2024-10-03-sequence_to_sequence_models_1/weights/attention/decoder.pt', map_location='cpu'))
	)
	encoder_decoder = EncoderDecoder(
		encoder=encoder,
		decoder=decoder
	)
	encoder_decoder.eval()
	
	return encoder_decoder

In [13]:
encoder_decoder = construct_model_attention()

In [14]:
os.environ['DEVICE'] = 'cpu'

In [30]:
test_sentences = ('I have socks.', 'I have socks that are thick and made of wool because it is cold outside; it is comfortable to wear around the house while I look outside at the white snow gently falling to the ground.')

for sent in test_sentences:
	input_ = source_tokenizer.processor.encode(sent)
	
	# add sos and eos tokens
	input_ = [source_tokenizer.processor.bos_id()] + input_ + [source_tokenizer.processor.eos_id()]
	
	sequence_generator = BeamSearchSequenceGenerator(
		encoder_decoder=encoder_decoder,
		tokenizer=target_tokenizer,
	)
	with torch.no_grad():
		pred = sequence_generator.generate(input_tensor=torch.tensor(input_), input_lengths=[len(input_)])
	
	# it returns list of top scoring beams. select best one, and get decoded text
	pred = pred[0][0]
	print(f'input: {sent}')
	print(f'pred: {pred}')
	print('\n')

input: I have socks.
pred: J'ai des chaussettes.

input: I have socks that are thick and made of wool because it is cold outside; it is comfortable to wear around the house while I look outside at the white snow gently falling to the ground.
pred: J'ai des chaussettes épais et faites de laine parce qu'il est froid à l'extérieur; il est confortable de m'habiller autour de la maison alors que je regarde à l'extérieur à la neige blanche qui tombe doucement au sol.
